In [1]:
# Code source: Brian McFee
# License: ISC

from __future__ import print_function

import numpy as np
import scipy
import matplotlib.pyplot as plt

import sklearn.cluster

import librosa
import librosa.display

In [2]:
file_path = "../data/songData/genres/blues/blues.00000.wav"
y, sr = librosa.load(file_path)

In [3]:
BINS_PER_OCTAVE = 12 * 3
N_OCTAVE = 7

C = librosa.amplitude_to_db(librosa.cqt(y=y, sr=sr,
                                        bins_per_octave=BINS_PER_OCTAVE,
                                        n_bins=N_OCTAVE * BINS_PER_OCTAVE),
                           ref=np.max)

plt.figure(figsize=(12, 4))
librosa.display.specshow(C, y_axis='cqt_hz', sr=sr,
                        bins_per_octave=BINS_PER_OCTAVE,
                        x_axis='time')
plt.title('log power cqt')
plt.tight_layout()

In [4]:
S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
log_S = librosa.logamplitude(S, ref_power=np.max)

plt.figure(figsize=(12, 4))
librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')
plt.title('mel power spectrogram')
plt.colorbar(format='%02.0f dB')
plt.tight_layout()

In [5]:
tempo, beats = librosa.beat.beat_track(y=y, sr=sr, trim=False)
Csync = librosa.util.sync(C, beats, aggregate=np.median)

beat_times = librosa.frames_to_time(librosa.util.fix_frames(beats,
                                                           x_min=0,
                                                           x_max=C.shape[1]),
                                   sr=sr)

plt.figure(figsize=(12, 4))
librosa.display.specshow(Csync, bins_per_octave=12*3,
                        y_axis='cqt_hz', x_axis='time',
                        x_coords=beat_times)
plt.tight_layout()